In [27]:
from bs4 import BeautifulSoup as bs
import requests as req
import pandas as pd

In [6]:
#basicamente baixo o html, extraio o texto dele e uso o BeautifulSoup para deixar mais limpo
meu_html = req.get("http://arquivos.proderj.rj.gov.br/sefaz_ceperj_imagens/Arquivos_Ceperj/ceep/dados-estatisticos/series-historicas/excel/cogeo/Tab%202.0.0.34.html")
meu_dado = meu_html.text
minha_sopa = bs(meu_dado)

In [7]:
minha_sopa

<!DOCTYPE html>

<html lang="pt-br">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<!-- Nome que vai aparecer na aba do navegador -->
<title>Series Historicas</title>
<meta content="Source code generated using layoutit.com" name="description"/>
<meta content="LayoutIt!" name="author"/>
<link href="css/bootstrap.min.css" rel="stylesheet"/>
<link href="css/bootstrap.css" rel="stylesheet"/>
<link href="css/style.css" rel="stylesheet"/>
</head>
<body>
<div class="container-fluid">
<div class="row topbar">
<div class="col-md-12 ml-0">
<img alt="Governo do Estado do Rio de Janeiro" class="ml-5" src="img/img-logo-gov-br-new.png"/>
</div>
</div>
<div class="row mb-3">
<div class="col-md-12 mt-3">
<img src="img/logo-2.jpg" width="100%"/>
</div>
</div>
<!-- inicio tabela -->
<div class="row">
<div class="col-md-12">
<table class="table table-bordered table-sm">
<thead>
<h3 class="mt-5">


In [108]:
#Começa aqui o webscrapping buscando as tabelas dentro do código html
minha_tabela = minha_sopa.find_all("table")[0] #o zero é um slicer, para selecionar a primeira tabela que tiver na página
lista = []

#Dentro da entrutura da página os dados estão contidos dentro de uma tag <tr> e depois dentro de uma <td>
#Basicamente vou buscar todas as DIVs que contem tr
for tr in minha_tabela.find_all("tr"):
    linha = []
    #depois vou pegar em cada DIV TR as DIVs td
    tds = tr.find_all("td")
    #se não tiver achado nada, e o a variável tds estiver vazia eu pulo a iteração
    if not tds:
        continue
    #se a td possuir valores, eu vou iterar em todas as linhas da DIV td, para pegar o valor e appendar na minha lista completa
    for td in tds:
        linha.append(td.text.strip()) #ao utilizar o .strip eu consigo limpar os separadores html, que são basicamente /r /n /b etc...
    lista.append(linha)
        
lista = lista[2:-1] #ao analisar a lista vimos que as duas primeiras linhas e a última linha eram lixo, por isso as retiramos

cabecalhos = ['IBGE','CIDADE','DIGIT','LAT','LONG']
dicionario = []

for i,x in enumerate(lista):
    dicionario += [dict(list(zip(cabecalhos,lista[i])))] #aqui eu começo a montar tuplas com o nome da coluna + valor

def LATLONG_CONVERTER(texto): #defino uma função que converte o formato original de lat/long para o formato de graus
    texto = texto.split()
    grau, minuto, segundo = [int(num) for num in texto[1::2]]
    graus = grau + minuto / 60 + segundo / (60*60)
    return graus

for i,x in enumerate(dicionario): #itero em cada registro do dicionario para fazer a conversão de lat/long
    if dicionario[i]['LAT'] == 'nd': #pulo municipios que estão sem lat/long, com o valor 'nd'
        continue
    dicionario[i]['LAT'] = LATLONG_CONVERTER(dicionario[i]['LAT'])
    dicionario[i]['LONG'] = LATLONG_CONVERTER(dicionario[i]['LONG'])

meu_df = pd.DataFrame(dicionario) #e por fim converto para um dataframe
meu_df

,IBGE,CIDADE,DIGIT,LAT,LONG
0,330010,Angra dos Reis,6,23.006667,44.318056
1,330015,Aperibé,71,21.620833,42.102778
2,330020,Araruama,15,22.872778,42.343056
3,330022,Areal,444,22.230556,43.105556
4,330023,Armação dos Búzios,3,22.746944,41.881667
...,...,...,...,...,...
88,330600,Três Rios,269,22.116667,43.209167
89,330610,Valença,560,22.245556,43.700278
90,330615,Varre-Sai,680,20.931111,41.868611
91,330620,Vassouras,434,22.403889,43.6625
